# 🚢 EPA Titanic KNN — Feature Engineering
KNN con ingeniería de atributos (FamilySize, IsAlone, Title, CabinKnown, FarePerPerson, TicketGroupSize) + preprocesamiento y Grid Search.

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


# 📥 Celda 2 — Carga de datos (train y test)
Leemos los CSV oficiales desde `/kaggle/input/titanic/` y validamos dimensiones.

In [5]:
# Importamos pandas para manejo de DataFrames
import pandas as pd

# Definimos rutas de los archivos provistos por la competición
train_path = "/kaggle/input/titanic/train.csv"   # ruta de train.csv
test_path  = "/kaggle/input/titanic/test.csv"    # ruta de test.csv

# Cargamos los datasets en memoria
train_df = pd.read_csv(train_path)               # DataFrame de entrenamiento
test_df  = pd.read_csv(test_path)                # DataFrame de test (sin Survived)

# Mostramos formas para confirmar tamaño esperado
print("Shape train:", train_df.shape)            # debería ser (891, 12)
print("Shape test :", test_df.shape)             # debería ser (418, 11)

# Vista rápida de columnas relevantes
train_df.head()                                  # primeras filas para inspección visual

Shape train: (891, 12)
Shape test : (418, 11)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# 🔎 Celda 3 — EDA breve (faltantes y balance de Survived)
Revisamos cuántos valores faltantes hay por columna en train y test, 
y verificamos la distribución de la variable objetivo `Survived`.

In [6]:
# Mostramos las columnas disponibles en el dataset de entrenamiento
print("Columnas en train:", train_df.columns.tolist())  

# Revisamos cuántos valores faltantes (NaN) hay por columna en train
print("\nFaltantes en train:")
print(train_df.isna().sum())  

# Revisamos cuántos valores faltantes (NaN) hay por columna en test
print("\nFaltantes en test:")
print(test_df.isna().sum())  

# Verificamos la distribución de la variable objetivo Survived (0 = no, 1 = sí)
print("\nDistribución de Survived en train:")
print(train_df["Survived"].value_counts(normalize=True))  

Columnas en train: ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']

Faltantes en train:
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Faltantes en test:
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

Distribución de Survived en train:
Survived
0    0.616162
1    0.383838
Name: proportion, dtype: float64


# 🧱 Celda 4 — Definición de variables base y objetivo
Separamos las columnas de entrada (`X`) y la variable objetivo (`y`) para el entrenamiento.
Además, retenemos el `PassengerId` del test para construir luego el archivo de submission.

In [7]:
# Definimos columnas numéricas base (antes de aplicar feature engineering)
base_num_features = ["Age", "SibSp", "Parch", "Fare", "Pclass"]

# Definimos columnas categóricas base
base_cat_features = ["Sex", "Embarked"]

# Construimos X con las columnas base desde train
X = train_df[base_num_features + base_cat_features].copy()

# Definimos el vector objetivo (y = Survived) y lo convertimos a entero
y = train_df["Survived"].astype(int)

# Guardamos PassengerId del test (necesario para el submission final)
test_passenger_id = test_df["PassengerId"].copy()

# Construimos X_test con las mismas columnas base desde test
X_test = test_df[base_num_features + base_cat_features].copy()

# Vista previa de las primeras filas de X
X.head()

,Age,SibSp,Parch,Fare,Pclass,Sex,Embarked
0,22.0,1,0,7.2500,3,male,S
1,38.0,1,0,71.2833,1,female,C
2,26.0,0,0,7.9250,3,female,S
3,35.0,1,0,53.1000,1,female,S
4,35.0,0,0,8.0500,3,male,S


# 🧪 Celda 5 — Ingeniería de atributos (FunctionTransformer)
Creamos nuevas features dentro de un `Pipeline`:  
- `FamilySize = SibSp + Parch + 1`  
- `IsAlone` (1 si viaja solo)  
- `Title` (extraído de `Name`)  
- `CabinKnown` (1 si `Cabin` no es NaN)  
- `FarePerPerson = Fare / FamilySize`  
- `TicketGroupSize` (tamaño de grupo por `Ticket`)  

> Nota: Para que el transformador pueda extraer títulos y usar `Ticket/Cabin`, **añadimos temporalmente** esas columnas a `X` y `X_test`. Luego, el `ColumnTransformer` las descartará (no están en las listas finales de features).

In [8]:
# --- Aseguramos que X y X_test incluyan columnas necesarias para FE (Name, Ticket, Cabin) ---
extra_cols = ["Name", "Ticket", "Cabin"]                                # columnas auxiliares para FE
for col in extra_cols:
    if col not in X.columns and col in train_df.columns:
        X[col] = train_df.loc[X.index, col]                             # añadimos desde train_df por índice
    if col not in X_test.columns and col in test_df.columns:
        X_test[col] = test_df.loc[X_test.index, col]                    # añadimos desde test_df por índice

# --- Definimos el transformador de FE ---
from sklearn.preprocessing import FunctionTransformer
import numpy as np
import pandas as pd

def add_engineered_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()                                                       # no modificar in-place

    # FamilySize e IsAlone
    df["FamilySize"] = df["SibSp"].fillna(0) + df["Parch"].fillna(0) + 1
    df["IsAlone"] = (df["FamilySize"] == 1).astype(int)

    # Title (si existe Name)
    if "Name" in df.columns:
        titles = df["Name"].str.extract(r",\s*([^\.]+)\.")[0]            # texto entre la coma y el punto
        titles = titles.replace({"Mlle": "Miss", "Ms": "Miss", "Mme": "Mrs"})
        common = {"Mr","Mrs","Miss","Master","Dr","Rev","Col","Major","Lady","Countess","Jonkheer","Don","Dona","Capt","Sir"}
        titles = titles.where(titles.isin(list(common)), "Other")
        df["Title"] = titles.fillna("Other")
    else:
        df["Title"] = "Other"

    # CabinKnown (si existe Cabin)
    if "Cabin" in df.columns:
        df["CabinKnown"] = (~df["Cabin"].isna()).astype(int)
    else:
        df["CabinKnown"] = 0

    # FarePerPerson
    df["FarePerPerson"] = (df["Fare"].fillna(df["Fare"].median())) / df["FamilySize"].replace(0, 1)

    # TicketGroupSize (si existe Ticket)
    if "Ticket" in df.columns:
        counts = df["Ticket"].map(df["Ticket"].value_counts())
        df["TicketGroupSize"] = counts.fillna(1).astype(int)
    else:
        df["TicketGroupSize"] = 1

    return df

# Creamos el FunctionTransformer para usarlo en el Pipeline
feature_engineering = FunctionTransformer(add_engineered_features, validate=False)

print("✅ Ingeniería de atributos lista (FunctionTransformer definido).")

✅ Ingeniería de atributos lista (FunctionTransformer definido).


# 🧽 Celda 6 — Preprocesamiento por tipo de columna
Imputación (mediana/moda), escalado (StandardScaler) y One-Hot para categóricas.
Incluye las **features ingenierizadas** generadas en la Celda 5.

In [9]:
# Importamos las piezas necesarias para el preprocesamiento y el pipeline
from sklearn.compose import ColumnTransformer                      # aplicar transformaciones por columnas
from sklearn.pipeline import Pipeline                              # encadenar pasos
from sklearn.impute import SimpleImputer                           # imputar NaN
from sklearn.preprocessing import OneHotEncoder, StandardScaler    # one-hot y escalado

# Definimos las columnas finales que usaremos DESPUÉS de la ingeniería de features
# (las columnas auxiliares Name/Ticket/Cabin NO se listan aquí; no serán usadas como features directas)
num_features = [
    "Age", "SibSp", "Parch", "Fare", "Pclass",  # base numéricas
    "FamilySize", "IsAlone", "FarePerPerson",   # FE numéricas
    "TicketGroupSize", "CabinKnown"             # FE numéricas/bool
]

cat_features = [
    "Sex", "Embarked", "Title"                  # categóricas (Title viene de FE)
]

# Pipeline para columnas numéricas: imputar con mediana + escalar
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),  # NaN -> mediana (robusto a outliers)
    ("scaler", StandardScaler())                    # requerido para distancias en KNN
])

# Pipeline para columnas categóricas: imputar con moda + One-Hot (denso)
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),             # NaN -> valor más frecuente
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))  # sin warnings, ignora categorías nuevas
])

# ColumnTransformer que aplica cada pipeline a su grupo de columnas
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_features),
        ("cat", categorical_transformer, cat_features)
    ],
    remainder="drop"  # descarta columnas no listadas (p. ej., Name/Ticket/Cabin auxiliares)
)

# Información útil para verificar
print("✅ Preprocesamiento listo.")
print("Num features:", num_features)
print("Cat features:", cat_features)

✅ Preprocesamiento listo.
Num features: ['Age', 'SibSp', 'Parch', 'Fare', 'Pclass', 'FamilySize', 'IsAlone', 'FarePerPerson', 'TicketGroupSize', 'CabinKnown']
Cat features: ['Sex', 'Embarked', 'Title']


# 🤖 Celda 7 — Pipeline completo (FE + Prepro + KNN) y validación cruzada
Encadenamos **ingeniería de atributos → preprocesamiento → KNN** en un solo `Pipeline`
y medimos el desempeño con **validación cruzada estratificada (5 folds)** usando *accuracy*.

In [10]:
# Importamos el clasificador KNN y utilidades para validación
from sklearn.neighbors import KNeighborsClassifier                 # modelo KNN
from sklearn.model_selection import StratifiedKFold, cross_val_score  # CV estratificada
import numpy as np                                                  # operaciones numéricas

# Definimos un KNN base como punto de partida (luego haremos Grid Search en la Celda 8)
base_knn = KNeighborsClassifier(
    n_neighbors=7,        # k inicial razonable
    weights="uniform",    # todos los vecinos pesan igual (alternativa: "distance")
    p=2                   # distancia Euclídea (p=1 sería Manhattan)
)

# Construimos el pipeline completo: Feature Engineering -> Preprocesamiento -> Modelo
pipe_knn = Pipeline(steps=[
    ("fe", feature_engineering),   # añade columnas ingenierizadas a partir de X
    ("pre", preprocessor),         # imputación, escalado y one-hot por tipo de columna
    ("knn", base_knn)              # clasificador KNN
])

# Definimos validación cruzada estratificada (mantiene proporción de clases)
cv = StratifiedKFold(
    n_splits=5,       # 5 folds
    shuffle=True,     # baraja antes de dividir
    random_state=42   # reproducibilidad
)

# Evaluamos el pipeline con accuracy mediante cross-validation
cv_scores = cross_val_score(
    estimator=pipe_knn,  # pipeline completo
    X=X,                 # características (con columnas base; FE se aplica dentro del pipeline)
    y=y,                 # variable objetivo
    cv=cv,               # esquema de validación
    scoring="accuracy",  # métrica oficial de la competición
    n_jobs=-1            # usa todos los núcleos disponibles
)

# Mostramos resultados por fold y resumen
print("Accuracy por fold:", np.round(cv_scores, 4))                   # accuracies individuales
print("Accuracy promedio (CV):", cv_scores.mean().round(4),           # promedio de accuracy
      "| Desv. std:", cv_scores.std().round(4))                       # variabilidad entre folds


Accuracy por fold: [0.8324 0.8202 0.7528 0.8202 0.8202]
Accuracy promedio (CV): 0.8092 | Desv. std: 0.0286


# 🔎 Celda 8 — Grid Search de KNN con Feature Engineering
Exploramos distintas combinaciones de hiperparámetros para KNN:  
- `n_neighbors` (k: cantidad de vecinos)  
- `weights` (uniforme vs. ponderado por distancia)  
- `p` (1 = Manhattan, 2 = Euclídea)  

El objetivo es encontrar la configuración que maximiza **accuracy** en validación cruzada.


In [11]:
# Importamos GridSearchCV para búsqueda exhaustiva
from sklearn.model_selection import GridSearchCV

# Reconstruimos el pipeline (por claridad) incluyendo FE + Prepro + KNN
pipe_knn = Pipeline(steps=[
    ("fe", feature_engineering),   # ingeniería de atributos
    ("pre", preprocessor),         # preprocesamiento
    ("knn", KNeighborsClassifier())# KNN "vacío" -> parámetros definidos por grid
])

# Definimos la grilla de hiperparámetros a explorar
param_grid = {
    "knn__n_neighbors": [3, 5, 7, 9, 11, 13, 15, 19, 25, 31],  # distintos valores de k
    "knn__weights": ["uniform", "distance"],                   # peso uniforme o por distancia
    "knn__p": [1, 2]                                           # distancia Manhattan (1) o Euclídea (2)
}

# Reutilizamos validación cruzada estratificada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Configuramos GridSearchCV
grid = GridSearchCV(
    estimator=pipe_knn,          # pipeline completo
    param_grid=param_grid,       # grilla de búsqueda
    scoring="accuracy",          # métrica oficial
    cv=cv,                       # validación cruzada
    n_jobs=-1,                   # usar todos los cores disponibles
    refit=True,                  # reentrena el mejor en todo el train
    verbose=1                    # verboso para seguir el progreso
)

# Ejecutamos el grid search
grid.fit(X, y)

# Reportamos los mejores resultados
print("🔝 Mejor accuracy (CV):", grid.best_score_.round(4))
print("🔧 Mejores hiperparámetros:", grid.best_params_)

# Guardamos el mejor pipeline ya reentrenado
best_model = grid.best_estimator_

# Mostramos los 5 mejores resultados del grid ordenados
res = pd.DataFrame(grid.cv_results_).sort_values("mean_test_score", ascending=False)
res_top5 = res[["mean_test_score","std_test_score","param_knn__n_neighbors","param_knn__weights","param_knn__p"]].head(5)
display(res_top5)


Fitting 5 folds for each of 40 candidates, totalling 200 fits
🔝 Mejor accuracy (CV): 0.8249
🔧 Mejores hiperparámetros: {'knn__n_neighbors': 13, 'knn__p': 1, 'knn__weights': 'uniform'}


,mean_test_score,std_test_score,param_knn__n_neighbors,param_knn__weights,param_knn__p
20,0.824895,0.020994,13,uniform,1
8,0.820394,0.023235,7,uniform,1
32,0.819302,0.002380,25,uniform,1
12,0.819289,0.014982,9,uniform,1
28,0.818178,0.009186,19,uniform,1


# 📤 Celda 9 — Predicción final y creación de `submission.csv`
Usamos el **mejor pipeline** encontrado por GridSearchCV (`best_model`) para predecir en `test.csv`
y generamos el archivo de envío con las columnas `PassengerId` y `Survived` (418 filas).


In [12]:
# 1) Predicción en el set de test usando el mejor modelo del grid (ya reentrenado con refit=True)
test_preds = best_model.predict(X_test)          # predicciones binarias 0/1
test_preds = test_preds.astype(int)              # aseguramos tipo entero

# 2) Construimos el DataFrame de submission con el formato oficial
submission = pd.DataFrame({
    "PassengerId": test_passenger_id,           # IDs del test
    "Survived": test_preds                      # predicciones
})

# 3) Chequeos de formato (defensivos)
assert submission.shape[0] == 418, "El submission debe tener exactamente 418 filas."
assert list(submission.columns) == ["PassengerId", "Survived"], "Las columnas deben ser exactamente PassengerId y Survived."

# 4) Guardamos el CSV listo para subir a Kaggle
submission.to_csv("submission.csv", index=False)

# 5) Confirmación y vista previa
print("✅ Archivo 'submission.csv' creado con", submission.shape[0], "filas.")
print(submission.head())
print("\nConteo predicciones:", submission["Survived"].value_counts().to_dict())


✅ Archivo 'submission.csv' creado con 418 filas.
   PassengerId  Survived
0          892         0
1          893         1
2          894         0
3          895         0
4          896         1

Conteo predicciones: {0: 246, 1: 172}


# 🏆 Resultado del Submission

- **Score Kaggle:** 0.75598  
- **Notebook:** EPA_TITANIC_KNN_FE_V1.ipynb  
- **Modelo:** KNN con Feature Engineering + GridSearch (k=13, p=1, uniform)  
- **Notas:**  
  - CV promedio: 0.8249  
  - Buen desempeño interno, pero peor generalización en Kaggle (posible sobreajuste de KNN a las nuevas features).  
  - Próximo paso: probar un modelo basado en árboles (Random Forest, Gradient Boosting) con estas mismas features.  
